In [1]:
%load_ext autoreload
%autoreload 2

In [76]:
from iexfinance.refdata import get_symbols, get_iex_symbols

import os
import sys
import pandas as pd
import altair as alt
import datetime as dt

from pandas import to_datetime
from datetime import datetime, date
from iexfinance.stocks import get_historical_data
from dateutil.relativedelta import relativedelta

sys.path.append('analysis/src')
os.environ["START_DATE"] = "2010-08-25"
os.environ["END_DATE"] = "2020-08-25"
from analysis.src.per_high_low import get_min_max_dfs, compute_near_max_min, save_daily_results

In [3]:
# Used to log the `MESSAGES USED` in `_IEXBAse`.
# https://addisonlynch.github.io/iexfinance/stable/logging.html
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")
os.environ["IEX_LOG_LEVEL"] = "DEBUG"

DEBUG:root:test


In [4]:
IEX_TOKEN = "Tpk_57fa15c2c86b4dadbb31e0c1ad1db895"
os.environ["IEX_API_VERSION"] = "iexcloud-sandbox"
store = pd.HDFStore('dev_iex_store.h5')

In [5]:
all_symbols = store.keys()

In [6]:
print(store['AAPL'].index.min())
print(store['AAPL'].index.max())
print(store['NTZ'].index.min())

2006-12-29 00:00:00
2020-12-24 00:00:00
2006-12-29 00:00:00


In [7]:
import random
RANDOM_SAMPLE_SIZE = 3
symbols = random.sample(all_symbols, RANDOM_SAMPLE_SIZE)
symbols = ['AAPL'] + [symbol[1:] for symbol in symbols]

In [28]:
dfs_res = compute_near_max_min(symbols)
dfs_res

AAPL: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
AAPL: Data is catched between 2010-08-25 00:00:00 and 2021-01-04 00:00:00.
Successfully processed AAPL
GCOR: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
GCOR: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Failed to process GCOR: Wrong number of items passed 4, placement implies 1.
IMMP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
IMMP: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Successfully processed IMMP
GJP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
GJP: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Successfully processed GJP
DONE computing min & max for 0.1
DONE computing min & max for 0.2


{0.1:             near_min  near_max
 date                          
 2010-08-25       NaN       NaN
 2010-08-30       NaN       NaN
 2010-08-31       NaN       NaN
 2010-09-01       NaN       NaN
 2010-09-02       NaN       NaN
 ...              ...       ...
 2020-08-07       0.0       1.0
 2020-08-13       0.0       1.0
 2020-08-17       0.0       1.0
 2020-08-21       0.0       1.0
 2020-08-25       0.0       1.0
 
 [1261 rows x 2 columns],
 0.2:             near_min  near_max
 date                          
 2010-08-25       NaN       NaN
 2010-08-30       NaN       NaN
 2010-08-31       NaN       NaN
 2010-09-01       NaN       NaN
 2010-09-02       NaN       NaN
 ...              ...       ...
 2020-08-07       1.0       1.0
 2020-08-13       1.0       1.0
 2020-08-17       1.0       1.0
 2020-08-21       1.0       1.0
 2020-08-25       1.0       1.0
 
 [1261 rows x 2 columns]}

In [29]:
# https://github.com/altair-viz/altair/issues/968
df_res = dfs_res[0.1]
df_alt = df_res.dropna(thresh=1)
df_alt = df_alt[df_alt.index>'2012-01-01']
data_alt = df_alt.reset_index().melt('date')

In [38]:
data_alt[data_alt['variable'] == 'near_min'].value.mean()

0.46633663366336636

In [56]:
# https://altair-viz.github.io/gallery/multiline_tooltip.html

import altair as alt
import pandas as pd

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['date'], empty='none')

# The basic line
line = alt.Chart(data_alt).mark_line(interpolate='basis').encode(
    x='date:T',
    y='value:Q',
#     color=alt.Color('color:N', scale = None)
    color='variable:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(data_alt).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(data_alt).mark_rule(color='gray').encode(
    x='date:T',
).transform_filter(
    nearest
)

# Horizontal average lines
avg_min = data_alt[data_alt['variable'] == 'near_min'].value.mean()
avg_max = data_alt[data_alt['variable'] == 'near_max'].value.mean()

line_avg_min = alt.Chart(pd.DataFrame({'value': [avg_min]})).mark_rule(strokeDash=[5,5]).encode(y='value')
line_avg_max = alt.Chart(pd.DataFrame({'value': [avg_max]})).mark_rule(strokeDash=[5,5]).encode(y='value')

# Put the five layers into a chart and bind the data
chart = alt.layer(
    line, selectors, points, rules, text, line_avg_min, line_avg_max
).properties(
    width=600, height=300
)


with alt.data_transformers.enable(max_rows=None):
    chart.display()
    #.save('chart.html', embed_options={'renderer':'svg'})

alt.LayerChart(...)

In [67]:
dfs_res = compute_near_max_min(symbols)

AAPL: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
AAPL: Data is catched between 2010-08-25 00:00:00 and 2021-01-04 00:00:00.
Successfully processed AAPL
GCOR: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
GCOR: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Failed to process GCOR: Wrong number of items passed 4, placement implies 1.
IMMP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
IMMP: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Successfully processed IMMP
GJP: Trying to access historical data between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
GJP: Data is catched between 2010-08-25 00:00:00 and 2020-08-25 00:00:00.
Successfully processed GJP
DONE computing min & max for 0.1
DONE computing min & max for 0.2


In [71]:
dfs_res[0.1]

,near_min,near_max
date,,
2010-08-25,NaN,NaN
2010-08-30,NaN,NaN
2010-08-31,NaN,NaN
2010-09-01,NaN,NaN
2010-09-02,NaN,NaN
...,...,...
2020-08-07,0.0,1.0
2020-08-13,0.0,1.0
2020-08-17,0.0,1.0


In [95]:
for delta in dfs_res.keys(): save_daily_results(dfs_res[delta], delta)

./dev_per_high_low_2021_01_04.html
Blob name must be an absolute path
Blob name must be an absolute path
./dev_per_high_low_2021_01_04.html
Blob name must be an absolute path
Blob name must be an absolute path
